# First aproach & Data exploration

In [516]:
!pip3 install pandas


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [517]:
import pandas as pd
import re

In [518]:
csv_file = "./the_office_lines_scripts.csv"
json_file = "./stopwords.json"

In [519]:
scripts_df = pd.read_csv(csv_file, encoding="utf_8")
scripts_df

,id,season,episode,scene,line_text,speaker,deleted
0,1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False
1,2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False
2,3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False
3,4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False
4,5,1,1,1,"All right. Well, let me show you how it's done.",Michael,False
...,...,...,...,...,...,...,...
59904,59905,9,23,112,It all seems so very arbitrary. I applied for ...,Creed,False
59905,59906,9,23,113,I just feel lucky that I got a chance to share...,Meredith,False
59906,59907,9,23,114,I���m happy that this was all filmed so I can ...,Phyllis,False
59907,59908,9,23,115,I sold paper at this company for 12 years. My ...,Jim,False


We would drop the deleted column as it won't be used in this activities.

In [520]:
scripts_df = scripts_df.drop(["deleted"], axis=1)
scripts_df

,id,season,episode,scene,line_text,speaker
0,1,1,1,1,All right Jim. Your quarterlies look very good...,Michael
1,2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim
2,3,1,1,1,So you've come to the master for guidance? Is ...,Michael
3,4,1,1,1,"Actually, you called me in here, but yeah.",Jim
4,5,1,1,1,"All right. Well, let me show you how it's done.",Michael
...,...,...,...,...,...,...
59904,59905,9,23,112,It all seems so very arbitrary. I applied for ...,Creed
59905,59906,9,23,113,I just feel lucky that I got a chance to share...,Meredith
59906,59907,9,23,114,I���m happy that this was all filmed so I can ...,Phyllis
59907,59908,9,23,115,I sold paper at this company for 12 years. My ...,Jim


In [521]:
stop_words_df = pd.read_json(json_file)
stop_words_df

,0
0,a
1,about
2,above
3,after
4,again
...,...
779,thorough
780,thoroughly
781,three
782,well


In [522]:
stop_words_df.describe()

,0
count,784
unique,781
top,keeps
freq,2


# Main data

For easy access to the information we can transform the scripts_df into three main variables: "characters_data" which is a dictionary capable of telling all the seasons/episodes in which a character was present, it also has a place for season metadata to solve the tasks of this option (words, stop words and lines); "stop_words": A list with all the stop words lowered and formatted so everything is consistent; "season_episodes": A dictionary with the seasons as keys and a list of episodes as data.


In [523]:
def add_character(name, data):
    if name not in data.keys():
        data[name] = {}
        
def add_season(season, name, data, seasons_episodes):
    if season not in data[name].keys():
        data[name][season] = {
            "episodes": [],
            "words": {},
            "n_words": 0,
            "n_lines": 0,
            "lines_per_ep": {},
            "stop_words": {},
            "n_stop_words": 0
        }
    if season not in seasons_episodes.keys():
        seasons_episodes[season] = {}
        
def add_episode(episode, season, name, data, seasons_episodes):
    if episode not in data[name][season]["episodes"]:
        data[name][season]["episodes"].append(episode)
        data[name][season]["lines_per_ep"][episode] = 0
    if episode not in seasons_episodes[season].keys():
        seasons_episodes[season][episode] = 1
    else:
        seasons_episodes[season][episode] += 1

In [524]:
def generate_main_data(scripts, stop):
    characters_data = {}
    seasons_episodes = {}
    stop_words = []
    for row in scripts.itertuples():
        add_character(row[6], characters_data)
        add_season(row[2], row[6], characters_data, seasons_episodes)
        add_episode(row[3], row[2], row[6], characters_data, seasons_episodes)

    for row in stop.itertuples():
        stop_words.append(re.sub('[^A-Za-z0-9]+', '', row[1]).lower())
    
    return characters_data, seasons_episodes, stop_words

# Tasks

## Create metadata

Using the main data we can now create the aditional information that will be used to solve the different questions.

In [525]:
def insert_word(word, stop_words, season_data):
    if word not in stop_words:
        if word in season_data["words"].keys():
            season_data["words"][word] += 1
            season_data["n_words"] += 1
        else:
            season_data["words"][word] = 1
            season_data["n_words"] += 1
    else:
        if word in season_data["stop_words"].keys():
            season_data["stop_words"][word] += 1
            season_data["n_stop_words"] += 1
        else:
            season_data["stop_words"][word] = 1
            season_data["n_stop_words"] += 1

def count_line(season_data, episode):
    season_data["n_lines"] += 1
    season_data["lines_per_ep"][episode] += 1

characters_data, season_episodes, stop_words = generate_main_data(scripts_df, stop_words_df)

for row in scripts_df.itertuples():
    season_data = characters_data[row[6]][row[2]]
    count_line(season_data, row[3])
    for word in row[5].strip().split():
        insert_word(re.sub('[^A-Za-z0-9]+', '', word).lower(), stop_words, season_data)

## Questions

Some questions are answered in this ipynb file and others are answered in a file that can be .txt or .csv. Also, the answer of the .csv files could be presented in only one file but for the assignment, I did it separately so each question has its answer.

### How many characters? What are their names?

"names.txt" has the list of names

In [526]:
n_characters = len(characters_data.keys())

with open('names.txt', 'w') as names_output:
    names_output.write(f"{n_characters} characters in this file.\n\n")
    count = 1
    for name in characters_data.keys():
        names_output.write(str(count) + " -> " + name + "\n")
        count += 1

print(f"there are {n_characters} characters in the series, their names are in \"names.txt\" file")

there are 793 characters in the series, their names are in "names.txt" file


### For each character, find out who has the most lines across all episodes

Michael, and he has 12140 lines across the series.

In [527]:
most_lines = ["", 0]
for name in characters_data.keys():
    total_lines = 0
    for season in characters_data[name].keys():
        total_lines += characters_data[name][season]["n_lines"]
    if total_lines > most_lines[1]:
        most_lines[0] = name
        most_lines[1] = total_lines
print(f"The charecter with the most lines across all the episodes is {most_lines[0]}, with {most_lines[1]} lines.")

The charecter with the most lines across all the episodes is Michael, with 12140 lines.


### What is the average of words per line for each character?

"avg_words_per_line.csv" has the answer.

In [528]:
with open('avg_words_per_line.csv', 'w') as avg_words_output:
    avg_words_output.write(f"Name, avg_words_per_line\n")
    for name in characters_data.keys():
        words = 0
        lines = 0
        for season in characters_data[name].keys():
            words += characters_data[name][season]["n_words"]
            lines += characters_data[name][season]["n_lines"]
        avg_words_output.write(f"{name},{words//lines}\n")

print("The answer is in avg_words_per_line.csv")

The answer is in avg_words_per_line.csv


### What is the most common word per character

"common_words.csv" has the answer.

In [529]:
def most_common_word(words):
    common = ["", 0]
    if len(words.keys()) == 0:
        return []
    for key in words.keys():
        if words[key] > common[1]:
            common[0] = key
            common[1] = words[key]
    return common

with open('common_words.csv', 'w') as common_words_output:
    common_words_output.write("name,most_common_word\n")
    for name in characters_data.keys():
        words_dict = {}
        for season in characters_data[name].keys():
            for key in characters_data[name][season]["words"].keys():
                if key not in words_dict.keys():
                    words_dict[key] = characters_data[name][season]["words"][key]
                else:
                    words_dict[key] += characters_data[name][season]["words"][key]
        words_dict = {k: v for k, v in sorted(words_dict.items(), key=lambda item: item[1], reverse=True)}
        common = most_common_word(words_dict)
        if len(common) == 0:
            common = ["Does not use words"]
        common_words_output.write(name + "," + common[0] + "\n")


print("The answer is in common_words.csv")    

The answer is in common_words.csv


### Number of episodes where the character does not have a line, for each character

"episodes_without_line.csv" has the answer.

In [530]:
with open('episodes_without_lines.csv', 'w') as no_lines_output:
    no_lines_output.write("name,number_of_episodes\n")
    for name in characters_data.keys():
        no_lines = 0
        for season in characters_data[name].keys():
            no_lines += (len(season_episodes[season].keys()) - len(characters_data[name][season]["episodes"]))
        no_lines_output.write(name + "," + str(no_lines) + "\n")

print("The answer is in episodes_without_line.csv")

The answer is in episodes_without_line.csv


### Number of times "That's what she said" joke comes up & five examples

There are 37 posible jokes, and the examples are in "jokes.txt". This file was created through trial an error until 5 jokes where obtained (the 1st and 2nd posible jokes weren't jokes, but usual dialog).

In [531]:
def get_posible_jokes(joke, scripts_data):
    posible = []
    joke = "That's what she said"
    for row in scripts_data.itertuples():
            if joke.lower() in row[5].lower():
                posible.append(row[0])
    return posible

posible_jokes = get_posible_jokes("That's what she said", scripts_df)
print(f"There are {len(posible_jokes)} posible instances of \"That´s what she said\" jokes.")


with open('jokes.txt', 'w') as jokes_output:
    count = 1
    for i in range(2, 8):
        jokes_output.write(f"Example {count}:\n")
        count += 1
        idx = posible_jokes[i] - 1
        while idx <= posible_jokes[i] + 1:
            jokes_output.write(scripts_df["speaker"].iloc[idx] + ": " + scripts_df["line_text"].iloc[idx] + "\n")
            idx += 1
        jokes_output.write("\n")

print("The examples are in jokes.txt and were discovered by printing some of the posible jokes ids (the first and second id weren't jokes)")

There are 37 posible instances of "That´s what she said" jokes.
The examples are in jokes.txt and were discovered by printing some of the posible jokes ids (the first and second id weren't jokes)


### The average percent of lines each character contributed to each episode per season

"lines_per_episode.csv" has the answer.

In [532]:
with open("lines_per_episode.csv", "w") as lines_per_episode:
    lines_per_episode.write("name, season, episode, percentage_spoken\n")
    for name in characters_data.keys():
            for season in characters_data[name].keys():
                for ep in characters_data[name][season]["episodes"]:
                    spoken = characters_data[name][season]["lines_per_ep"][ep]
                    total = season_episodes[season][ep]
                    lines_per_episode.write(name + "," + str(season) + "," + str(ep) + "," + str(spoken*100/total) + "\n")

print("The answer is in lines_per_episode.csv")

The answer is in lines_per_episode.csv


## 3 questions inveted by me

### Question 1: Per season, wich is the episode/s with more scenes?

"most_scenes.txt" has the answer.

In [533]:
with open('most_scenes.txt', 'w') as scenes_output:
    for season in season_episodes.keys():
        more_scenes = 0
        episodes_list = []
        for ep in season_episodes[season].keys():
            n_scenes = len(scripts_df[(scripts_df["season"] == season) & (scripts_df["episode"] == ep)].groupby(["scene"]).count())
            if n_scenes > more_scenes:
                more_scenes = n_scenes
                episodes_list = [ep]
            elif n_scenes == more_scenes:
                episodes_list.append(ep)
        scenes_output.write(f"The most scenes in Season {season} are {more_scenes} scenes and they are in the next episode/s:\n")
        for ep in episodes_list:
            scenes_output.write(f"Episode: {ep}\n")
        scenes_output.write("\n")
print("The answer is in the \"most_scenes.txt\" file.")

The answer is in the "most_scenes.txt" file.


### Question 2: Who is the character that uses the most stop words in the series?

Michael has the most stop words used in the series and he uses a total of 121190 stop words (times used not different stop words).

In [534]:
most_stop_words = ["", 0]
for name in characters_data.keys():
    stop_words_used = 0
    for season in characters_data[name].keys():
        for key in characters_data[name][season]["stop_words"].keys():
            stop_words_used += characters_data[name][season]["stop_words"][key]
    if stop_words_used > most_stop_words[1]:
        most_stop_words = [name, stop_words_used]
print(f"The charecter with the most stop words used across all the episodes is {most_stop_words[0]}, with {most_stop_words[1]} stop words used.")

The charecter with the most stop words used across all the episodes is Michael, with 121190 stop words used.


### Question 3: Most common stop word per character?

"common_stop_words.csv" has the answer.

In [535]:
with open('common_stop_words.csv', 'w') as stop_words_output:
    stop_words_output.write("name,most_common_stop_word\n")
    for name in characters_data.keys():
        words_dict = {}
        for season in characters_data[name].keys():
            for key in characters_data[name][season]["stop_words"].keys():
                if key not in words_dict.keys():
                    words_dict[key] = characters_data[name][season]["stop_words"][key]
                else:
                    words_dict[key] += characters_data[name][season]["stop_words"][key]
        words_dict = {k: v for k, v in sorted(words_dict.items(), key=lambda item: item[1], reverse=True)}
        common = most_common_word(words_dict)
        if len(common) == 0:
            common = ["Does not use stop words"]
        stop_words_output.write(name + "," + common[0] + "\n")


print("The answer is in common_stop_words.csv") 

The answer is in common_stop_words.csv


## Additional questions

### What are the most critical challenges for Adara, related to data science?

starting in Data science has many problems, but I believe these are the main 2 problems Adara could find according to its current state:

<li>Obtaining new data: Acquire new data is always hard especially when one is just starting, as Diego told me in the first meeting Adara is getting its data through polls on Instagram, so this means it's open to problems like having one user answer multiple times using different accounts which can lead to repeated data or non-representative data, being dependant of the store popularity as it's the buyers of the store the ones that fill out the polls and many others. I think that with the right tools like Instagram analytics (feature included in business accounts), and store data (most bought items, clicks on page ...etc) the collection of data could be improved and adjusted to have more of it and with fewer outliers. <br><br>

<li>Obteining unbiased data: This problem is a derivative of the previous one because if Adara is only using one source of information to obtain its data there is a higher chance that this data is biased towards the store's usual client. For example, if a new client approaches the system and its style is different than most clients he/she could be pushed away because the recommendations don't adjust to his/her taste. Maybe including other sources of data or techniques could adjust the system to include new clients (weighted models, using other store clients to fill the Adaras poll ...etc).

### Why did you choose this assignment instead of option 1?

I choose this assignment because it's been a long time since I was able to do "data analysis", so I wanted to see how I was doing and what I remembered from classes (Data analysis was one of my two minors, but because of curriculum problems I had to drop it). Also when people are trying to apply for a computer science job, usually, there's a Frontend/Backend assignment to solve and since I have been applying to different jobs I wanted to do something different to get out of the routine and to remember past knowledge (especially still being in University and working in Frontend or Backend in almost all my courses).